In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce 

# LOAD DATA

In [2]:
# Load pickle file with the categorical features selected
import pickle
with open('C://Users/luisgasco/Documents/bosh_kaggle_comp/data/list_sel_cat_feat_10k.pkl', 'rb') as f:
    cat_features = pickle.load(f)

Read the a sample of the rest of the dataset:

In [3]:
date_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_date.csv", chunksize=100000, dtype=np.float32)
num_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_numeric.csv",
                         usecols=list(range(970)), chunksize=100000, dtype=np.float32)
cat_chunks = pd.read_csv("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/train_categorical.csv", usecols = cat_features, chunksize=100000)
X = pd.concat([pd.concat([dchunk, nchunk, cchunk], axis=1).sample(frac=0.05)
               for dchunk, nchunk, cchunk in zip(date_chunks, num_chunks, cat_chunks)])


C:\Users\luisgasco\.conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (2,3,4,6,7,8,9,10,21,44,46,76,82,99,208,233,357,369,396,875,883,1022,1066,1081,1095,1096,1147,1184,1239,1321,1421,1431,1678,1693,1985,1987,2002) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\luisgasco\.conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (2,3,4,6,7,8,9,10,21,44,46,76,82,99,152,208,233,357,369,396,520,592,595,655,708,726,760,875,883,1022,1095,1147,1184,1239,1321,1421,1431,1678,1693,1985,1987,2002) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\luisgasco\.conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (2,3,4,6,7,8,9,10,21,44,46,76,82,99,152,233,357,369,371,396,580,592,595,65

In [4]:
y = X.Response
y.head()

52933    0.0
30367    0.0
19698    0.0
95176    0.0
95642    0.0
Name: Response, dtype: float32

In [5]:
X = X.drop(["Id","Response"],axis=1)

In [6]:
X.shape

(59187, 2199)

# Transformation of categorical features

Since we plan to train models such as XGBoost and RandomForest, we need to transform the categorical variables in the same way we do in previous notebooks

Load the previously created encoder:

In [100]:
import category_encoders as ce   # version 1.2.8
filename = "C://Users/luisgasco/Documents/bosh_kaggle_comp/data/encoder_categorical_model_FULL_FINAL_TRUE.sav"
ce_target = pickle.load(open(filename, 'rb'))
ce_target

LeaveOneOutEncoder(cols=['L1_S24_F726', 'L1_S25_F2024', 'L2_S28_F3285',
                         'L1_S24_F1559', 'L0_S1_F29', 'L1_S24_F1646',
                         'L0_S1_F27', 'L3_S35_F3912', 'L1_S24_F1144',
                         'L1_S24_F1523', 'L0_S2_F41', 'L1_S25_F2000',
                         'L1_S25_F2552', 'L1_S24_F936', 'L0_S1_F31',
                         'L0_S10_F220', 'L1_S24_F1584', 'L0_S11_F313',
                         'L1_S24_F1121', 'L1_S24_F1424', 'L1_S24_F1427',
                         'L1_S24_F1462', 'L0_S9_F183', 'L1_S24_F1152',
                         'L0_S2_F37', 'L0_S3_F79', 'L1_S24_F957',
                         'L1_S24_F1340', 'L0_S23_F674', 'L1_S25_F2047', ...],
                   sigma=0.1, verbose=1)

In [101]:
cat_fet = X.loc[:, cat_features]
cat_fet.shape

(59187, 75)

In [102]:
%%time
# Transform 
cat_fet_trans = ce_target.transform(cat_fet)

Wall time: 4.41 s


In [103]:
cat_fet_trans.shape

(59187, 75)

Concat again both the transformed categorical features with the rest of the dataframe:

In [104]:
X_otros =  X.drop(cat_features, axis=1)
X_end = pd.concat([X_otros,cat_fet_trans],axis=1)
X_end.shape

(59187, 2199)

We have 2199 features. We need to reduce the feature space prior to fine tune a model. For that reason we are going to select features by its importance by using XGBoost classifier.

In [65]:
from sklearn.model_selection import train_test_split, StratifiedKFold

X2 = X_end.values
y = y.ravel()

clf = XGBClassifier(learning_rate=0.1, max_depth=3,n_estimators=100,min_child_weight=1, nthread=-1)
cv = StratifiedKFold(n_splits=3)
preds = np.ones(y.shape[0])
for (train, test) in cv.split(X, y):
    preds[test] = clf.fit(X2[train], y[train]).predict_proba(X2[test])[:,1]
    print(" ROC AUC: {:.3f}".format( roc_auc_score(y[test], preds[test])))
print(roc_auc_score(y, preds))

 ROC AUC: 0.702
 ROC AUC: 0.649
 ROC AUC: 0.655
0.6681381084859601


In [114]:
most_important_features = pd.Series(clf.feature_importances_, index=X_end.columns).nlargest(100)
most_important_features

L3_S29_F3336    0.012911
L3_S48_F4193    0.012609
L0_S9_F170      0.011609
L3_S32_F3854    0.010966
L3_S30_F3819    0.010442
                  ...   
L3_S32_D3852    0.005547
L3_S30_F3579    0.005513
L0_S0_F0        0.005479
L1_S24_F1609    0.005469
L3_S36_F3920    0.005379
Length: 100, dtype: float32

In [124]:
df_importance = pd.DataFrame({'feature_name':X_end.columns, 'importance':clf.feature_importances_})
df_importance = df_importance.sort_values(by=['importance'], ascending=False)
df_importance_150 = df_importance[:150] 
df_importance_150.shape

(150, 2)

In [125]:
df_importance_150.to_pickle("C://Users/luisgasco/Documents/bosh_kaggle_comp/data/final_selected_features.pkl")